## PT complex tensor examples

In [1]:
import numpy as np
import torch
from pytorch_complex_tensor import ComplexTensor
import tensorflow as tf
tf.enable_eager_execution()

---   
Creation

In [2]:
# numpy complex tensor
np_c = np.asarray([[1+3j, 1+3j, 1+3j], [2+4j, 2+4j, 2+4j]]).astype(np.complex64)
np_c

array([[1.+3.j, 1.+3.j, 1.+3.j],
       [2.+4.j, 2.+4.j, 2.+4.j]], dtype=complex64)

In [3]:
# torch equivalent
pt_c = ComplexTensor([[1, 1, 1], [2,2,2], [3,3,3], [4,4,4]])
print(pt_c)

tensor([['(1.0+3.0j)' '(1.0+3.0j)' '(1.0+3.0j)'],
        ['(2.0+4.0j)' '(2.0+4.0j)' '(2.0+4.0j)']])


In [4]:
# verify reals match
print(np_c.real)
print(pt_c.real)

[[1. 1. 1.]
 [2. 2. 2.]]
tensor([[1., 1., 1.],
        [2., 2., 2.]])


In [5]:
# verify imag match
print(np_c.imag)
print(pt_c.imag)

[[3. 3. 3.]
 [4. 4. 4.]]
tensor([[3., 3., 3.],
        [4., 4., 4.]])


---   
Verify complex addition

In [6]:
np_c + (3+2j)

array([[4.+5.j, 4.+5.j, 4.+5.j],
       [5.+6.j, 5.+6.j, 5.+6.j]], dtype=complex64)

In [7]:
pt_c + (3 + 2j)

tensor([['(4.0+5.0j)' '(4.0+5.0j)' '(4.0+5.0j)'],
        ['(5.0+6.0j)' '(5.0+6.0j)' '(5.0+6.0j)']])

--- 
verify abs

In [8]:
np.abs(np_c)

array([[3.1622777, 3.1622777, 3.1622777],
       [4.472136 , 4.472136 , 4.472136 ]], dtype=float32)

In [9]:
pt_c.abs()

tensor([[3.1623, 3.1623, 3.1623],
        [4.4721, 4.4721, 4.4721]])

--- 
verify complex vs real matrix multiply

In [10]:
np_x = np.asarray([[3, 3], [4, 4], [2, 2]])
pt_x = torch.Tensor([[3, 3], [4, 4], [2, 2]])

print(np_x)
print(pt_x)

[[3 3]
 [4 4]
 [2 2]]
tensor([[3., 3.],
        [4., 4.],
        [2., 2.]])


In [11]:
np_mm_out = np.matmul(np_c, np_x)
np_mm_out

array([[ 9.+27.j,  9.+27.j],
       [18.+36.j, 18.+36.j]])

In [12]:
pt_mm_out = pt_c.mm(pt_x)
pt_mm_out

tensor([['(9.0+27.0j)' '(9.0+27.0j)'],
        ['(18.0+36.0j)' '(18.0+36.0j)']])

In [13]:
# verify reals
print(np_mm_out.real)
print(pt_mm_out.real)

[[ 9.  9.]
 [18. 18.]]
tensor([[ 9.,  9.],
        [18., 18.]])


In [14]:
# verify imags
print(np_mm_out.imag)
print(pt_mm_out.imag)

[[27. 27.]
 [36. 36.]]
tensor([[27., 27.],
        [36., 36.]])


--- 
verify transpose

In [15]:
np_c.T

array([[1.+3.j, 2.+4.j],
       [1.+3.j, 2.+4.j],
       [1.+3.j, 2.+4.j]], dtype=complex64)

In [16]:
pt_c.t()

tensor([['(1.0+3.0j)' '(2.0+4.0j)'],
        ['(1.0+3.0j)' '(2.0+4.0j)'],
        ['(1.0+3.0j)' '(2.0+4.0j)']])

--- 
## wfalcon/pytorch-complex-tensor


In [17]:
pt_c2 = ComplexTensor([[1, 3, 5], [7,9,11], [2,4,6], [8,10,12]])
print(pt_c2)
pt_c2.requires_grad = True

tensor([['(1.0+2.0j)' '(3.0+4.0j)' '(5.0+6.0j)'],
        ['(7.0+8.0j)' '(9.0+10.0j)' '(11.0+12.0j)']])


In [18]:
out = pt_c2 + 4
out = out.mm(pt_c2.t())
print(out)

tensor([['(15.0+136.0j)' '(69.0+334.0j)'],
        ['(-3.0+262.0j)' '(51.0+676.0j)']])


In [19]:
real_sum = out.real.sum()
print(real_sum)
out_imag = out.imag.sum()
print(out_imag)

tensor(132., grad_fn=<SumBackward0>)
tensor(1408., grad_fn=<SumBackward0>)


In [20]:
real_sum.backward()

In [21]:
pt_c2.grad

tensor([['(24.0-20.0j)' '(32.0-28.0j)' '(40.0-36.0j)'],
        ['(24.0-20.0j)' '(32.0-28.0j)' '(40.0-36.0j)']])

---  
### Use Tensorflow to compute grads


In [6]:
tf_c2 = tf.constant([[1+2j, 3+4j, 5+6j], [7+8j,9+10j,11+12j]], dtype=tf.complex64)

In [7]:
with tf.GradientTape() as t:
    t.watch(tf_c2)
    tf_out = tf_c2 + 4
    tf_out = tf.matmul(tf_out, tf.transpose(tf_c2, perm=[1,0]))
    print(tf_out)
    
    tf_y = tf.reduce_sum(tf_out)
    print(tf_y)

tf.Tensor(
[[15.+136.j 69.+334.j]
 [-3.+262.j 51.+676.j]], shape=(2, 2), dtype=complex64)
tf.Tensor((132+1408j), shape=(), dtype=complex64)


In [8]:
dy_dc2 = t.gradient(tf_y, tf_c2)

In [9]:
dy_dc2

<tf.Tensor: id=62, shape=(2, 3), dtype=complex64, numpy=
array([[24.-20.j, 32.-28.j, 40.-36.j],
       [24.-20.j, 32.-28.j, 40.-36.j]], dtype=complex64)>